# Implementation of MSM

Start by copying the code implementing consumption-savings model from  `lab5/egm.ipynb`, and make the following modifications:
- move `solve_egm` into the `deaton` model class to make it a class method
    * make sure to adjust the indent in each line!
    * effectively no change to the code is needed, only perhaps change first argument from *model* to *self*, yet the exact word used does not matter, so it should work as is
- also move `solve_plot`, `euler_residual`, `accuracy`, and `simulator` into the class in exactly the same way


## Consumption savings model from egm_pre.ipynb

In [4]:
# Copy the modified consumption-savings model here

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.stats import lognorm


class deaton():
  '''Implementation of the stochastic Deaton consumption-savings problem with random income.'''

  def __init__(self, Mbar=10, ngrid=50, nquad=10, interpolation='linear', beta=.9, R=1.05, sigma=1.):
    '''Object creator for the stochastic consumption-savings model'''
    self.beta = beta                        # Discount factor
    self.R = R                              # Gross interest
    self.sigma = sigma                      # Param in log-normal income distribution
    self.Mbar = Mbar                        # Upper bound on wealth
    self.ngrid = ngrid                      # Number of grid points in the state space
    self.nquad = nquad                      # Number of quadrature points
    self.interpolation = interpolation      # type of interpolation, see below
                                            # state and choice space grids, as well as quadrature points and weights are set with setter functions below

  def __repr__(self):
    '''String representation for the model'''
    return 'Deaton model with beta={:1.3f}, sigma={:1.3f}, gross return={:1.3f}\nGrids: state {} points up to {:1.1f}, quadrature {} points\nThe model is {}solved.'\
           .format(self.beta,self.sigma,self.R,self.ngrid,self.Mbar,self.nquad,'' if hasattr(self,'solution') else 'not ')

  @property
  def ngrid(self):
    '''Property getter for the ngrid parameter'''
    return self.__ngrid

  @ngrid.setter
  def ngrid(self, ngrid):
    '''Property setter for the ngrid parameter'''
    self.__ngrid = ngrid
    epsilon = np.finfo(float).eps                      # smallest positive float number difference
    self.grid = np.linspace(epsilon, self.Mbar, ngrid) # grid for state space

  @property
  def sigma(self):
    '''Property getter for the sigma parameter'''
    return self.__sigma

  @sigma.setter
  def sigma(self, sigma):
    '''Property setter for the sigma parameter'''
    self.__sigma = sigma
    self.__quadrature_setup()     # update quadrature points and weights

  @property
  def nquad(self):
    '''Property getter for the number of quadrature points'''
    return self.__nquad

  @nquad.setter
  def nquad(self, nquad):
    '''Property setter for the number of quadrature points'''
    self.__nquad = nquad
    self.__quadrature_setup()     # update quadrature points and weights

  def __quadrature_setup(self):
    '''Internal function to set up quadrature points and weights,
        depends on sigma and nquad, therefore called from the property setters
        '''
    try:
      # quadrature points and weights for log-normal distribution
      self.quadp, self.quadw = np.polynomial.legendre.leggauss(self.__nquad) # Gauss-Legendre for [-1,1]
      self.quadp = (self.quadp + 1) / 2                                      # rescale to [0,1]
      self.quadp = lognorm.ppf(self.quadp, self.__sigma)                     # inverse cdf
      self.quadw /= 2                                                        # rescale weights as well
    
    except (AttributeError):
      # when __nquad or __sigma are not yet set
      pass

  def utility(self, c):
    '''Utility function'''
    # logit utility
    return np.log(c)

  def marginal_utility(self, c):
    '''Marginal utility function'''
    return 1/c

  def inverse_marginal_utility(self, u):
    '''Marginal utility function'''
    return 1/u     # used for EGM step 4

  def next_period_wealth(self, M, c, y):
    '''Next period budget'''
    if self.nquad > 1: # for reference, intial default is 10, so ten points we use to approximate?
      # stochastic income case
      return self.R*(M-c)+y                       # next period wealth
    else:
      # use np.zeros(shape=y.shape) in place of y to revert the nquad=0 case for no income
      return self.R * (M - c) + np.zeros(shape=y.shape) # next period wealth without income

  def interp_func(self, x, f):
    '''Returns the interpolation function for given data'''
    if self.interpolation == 'linear':
      return interpolate.interp1d(x, f, kind='slinear', fill_value="extrapolate")
    elif self.interpolation == 'quadratic':
      return interpolate.interp1d(x, f, kind='quadratic', fill_value="extrapolate")
    elif self.interpolation == 'cubic':
      return interpolate.interp1d(x, f, kind='cubic', fill_value="extrapolate")
    elif self.interpolation == 'polynomial':
      p = np.polynomial.polynomial.polyfit(x, f, self.ngrid_state - 1)
      return lambda x: np.polynomial.polynomial.polyval(x, p)
    else:
      print('Unknown interpolation type')
      return None


In [5]:
# Run the following tests to see that the modelling object works as intended
# - initialize the model
model=deaton()
print(model)
# - explore how quadrature points and weights change with sigma and nquad
print('Quadrature points and weights (baseline)',model.quadp,model.quadw,sep='\n')
model2=deaton(nquad=20)
print(f'Quadrature points and weights (nquad={model2.nquad})',model2.quadp,model2.quadw,sep='\n')
model3=deaton(sigma=2.0)
print(f'Quadrature points and weights (sigma={model3.sigma})',model3.quadp,model3.quadw,sep='\n')


Deaton model with beta=0.900, sigma=1.000, gross return=1.050
Grids: state 50 points up to 10.0, quadrature 10 points
The model is not solved.
Quadrature points and weights (baseline)
[0.10808711 0.22426767 0.37037277 0.56379802 0.82887795 1.20645023
 1.77368483 2.6999825  4.45895742 9.25179704]
[0.03333567 0.07472567 0.10954318 0.13463336 0.14776211 0.14776211
 0.13463336 0.10954318 0.07472567 0.03333567]
Quadrature points and weights (nquad=20)
[ 0.06700331  0.1228722   0.18135407  0.24607771  0.3193444   0.40338206
  0.50072989  0.61449788  0.74865283  0.90841015  1.10082434  1.33573261
  1.6273449   1.99708468  2.47903933  3.1314155   4.06375694  5.51407521
  8.1385372  14.92463562]
[0.008807   0.02030071 0.03133602 0.04163837 0.05096506 0.05909727
 0.06584432 0.07104805 0.07458649 0.07637669 0.07637669 0.07458649
 0.07104805 0.06584432 0.05909727 0.05096506 0.04163837 0.03133602
 0.02030071 0.008807  ]
Quadrature points and weights (sigma=2.0)
[1.16828232e-02 5.02959887e-02 1.3717

## Simulation

In [ ]:
# Simulate a dataset that will be used for estimation
# (and check that the model code is working correctly)
model = deaton(beta=0.92,Mbar=50,ngrid=100)
model.solve_egm()
np.random.seed(14325)      # fix seed for initial draws
Nobs = 500 # number of observations to simulate
nT = 10 # number of time periods to simulate
init_wealth = np.exp(np.random.randn(Nobs)) # draw initial wealth
np.random.seed(15920) # fix seed for simulations
data = model.simulator(init_wealth=init_wealth,T=nT)
data_wealth = data['M']

NameError: name 'deaton' is not defined

In [ ]:
def moments_function(data, tail=10):
  '''Computes moments from
       the tail of the given time series (from last axis)
       Returns two vectors with moments computed at individual level
    '''
  d = data.ndim - 1     # last dimension
  mean = np.mean(data[:, -tail:], axis=d)
  std = np.std(data[:, -tail:], axis=d)
  ??? add more moments ???
  return mean, std

In [22]:
# Data moments
# NOTE: adjust the code according to the moment generating function!
print('Number of observed individuals: ',data_wealth.shape[0],sep=' ')
print('Number of observed time periods:',data_wealth.shape[1],sep=' ')
data_moment1, data_moment2 = moments_function(data_wealth)  # data moments on individual level (of observed sample)
data_moment1_mean, data_moment1_std = np.mean(data_moment1), np.std(data_moment1)  # descriptive stats for empirical moments
data_moment2_mean, data_moment2_std = np.mean(data_moment2), np.std(data_moment2)
print(f'Moment 1 (mean wealth), mean and std.dev. over data sample  : {data_moment1_mean:.5f} ({data_moment1_std:.5f})')
print(f'Moment 2 (std of wealth), mean and std.dev. over data sample: {data_moment2_mean:.5f} ({data_moment2_std:.5f})')
data_moments_vec = np.array([data_moment1_mean, data_moment2_mean]) # vector of aggregated moments

In [ ]:
# MSM estimation exercise
model = deaton(beta=0.95,Mbar=50,ngrid=100)  # init the model
np.random.seed(214)                          # fix for initial wealth
init_wealth = np.exp(np.random.randn(200))   # draw initial wealth, could be useful to use actual data from population of interest here (or sample thereof)

def moment_conditions(theta,data_moments,seed=215):
    '''Moment conditions for MSM estimator,
       Inputs: parameter vector + vector of aggregated data moments
       Computed at the individual level.
       Random number generator seed fixed by default.
    '''
    model.beta = theta
    np.random.seed(seed) # must be fixed between calls!
    model.solve_egm(maxiter=1000)
    simdata = model.simulator(init_wealth=init_wealth, T=60, plot=False)
    # compute simulated moments
    sim_moment1, sim_moment2 = moments_functions(simdata['M'])
    # return moment conditions
    return sim_moment1-data_moments[0] + ... ???

In [24]:
from scipy.optimize import minimize_scalar
from scipy.misc import derivative

def run_MSM(data_moments = data_moments_vec,                 # vector of data moments
            moment_conditions_function = moment_conditions,  # moment conditions generator
            W = None,                                        # weighting matrix
            bracket = [.85,.95],                             # approximate limits for the parameters
            plot = True):
    '''Run the MSM estimation
       Returns estimates and std.err. of estimates
    '''

    def mean_conditions(theta):
        '''Means of the moment conditions returned as a vector'''
        moms = moment_conditions_function(theta,data_moments)  # return a tuple
        moms = np.array(moms)  # convert to array, moments in axis=0, indiv in axis=1
        return np.mean(moms,axis=1)  # vector of means of moment conditions

    def criterion(theta,W):
        '''Criterion function for MSM estimator'''
        err = mean_conditions(theta)
        return err @ W @ err.T

    if W is None:
        # default weighting matrix = identity
        check = moment_conditions_function(1.0,data_moments)  # check how many moments
        W = np.eye(len(check))

    # minimize the criterion function
    res = minimize_scalar(criterion,method='Brent',args=(W),bracket=bracket,tol=1e-8)
    if not res.success:
        raise RuntimeError('Bellman continuous failed to find optimal consumption')
    theta_hat = res.x  # estimate

    # find out how many simulations were used
    moms = moment_conditions_function(theta_hat,data_moments)
    nsims = len(moms[0])  # will use in place of tau, assuming nobs=1 in the data

    D = derivative(mean_conditions,theta_hat,dx=1e-10)  # Jacobian of moment conditions
    DWD = D @ W @ D
    if np.isscalar(DWD):
        Sigma_hat = (1+1/nsims)/( DWD)  # using simple formula
        stderr = np.sqrt(Sigma_hat)
    else:
        Sigma_hat = (1+1/nsims)*np.linalg.inv( DWD)  # using simple formula
        stderr = np.sqrt(np.diag(Sigma_hat))
    CI = [theta_hat-1.96*stderr,theta_hat+1.96*stderr]  # 1.96 confidence interval

    print(f'MSM estimate       : {theta_hat:1.5f}')
    print(f'StdErr of estimate : {stderr:1.5f}')
    print(f'Confidence interval: ({CI[0]:1.5f},{CI[1]:1.5f})')

    if plot:
        # Plot criterion for visual inspection
        xd = np.linspace(bracket[0],bracket[1],50)
        yd = [criterion(b,W) for b in xd]
        fig,ax = plt.subplots(figsize=(12,8))
        ax.plot(xd,yd,color='r',label='MSM criterion function')
        y1,y2 = ax.get_ylim()
        ax.plot([theta_hat,theta_hat],[y1,y2],color='grey',label='MSM estimate')
        ax.fill_between(x=CI,y1=y1,y2=y2,color='grey',alpha=0.25,label='1.96 confidence interval')
        ax.legend()
        ax.set_title('Criterion function and MSM estimate');

    return theta_hat

In [25]:
beta_hat = run_MSM()  # initial run with default identity weighting matrix

In [19]:
# Second stage MSM
moms = moment_conditions(beta_hat,data_moments_vec,seed=515)  # simulate a separate set of moment conditions
S = np.cov(np.array(moms))                                    # variance-covariance matrix of moment conditions
W1 = np.linalg.inv(S)                                         # unpdated weighting matrix
beta_hat_2 = run_MSM(W=W1)